## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
sys.path.append('..')
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Buala,SB,-8.1450,159.5921,79.65,81,97,2.91,overcast clouds
1,1,Cape Town,ZA,-33.9258,18.4232,58.08,44,40,17.27,scattered clouds
2,2,Bambanglipuro,ID,-7.9500,110.2833,76.42,79,86,4.83,overcast clouds
3,3,Tecoanapa,MX,16.5167,-98.7500,86.81,65,36,3.27,scattered clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,44.22,56,75,6.91,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp) & (city_data_df['Max Temp']> min_temp)]


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Buala,SB,79.65,overcast clouds,-8.1450,159.5921,
2,Bambanglipuro,ID,76.42,overcast clouds,-7.9500,110.2833,
3,Tecoanapa,MX,86.81,scattered clouds,16.5167,-98.7500,
7,Basco,PH,83.48,light rain,20.4487,121.9702,
10,San-Pedro,CI,79.48,broken clouds,4.7485,-6.6363,
21,Bella Union,UY,76.21,overcast clouds,-30.2752,-57.6007,
24,Butaritari,KI,82.49,overcast clouds,3.0707,172.7902,
29,Ribeira Grande,PT,75.60,broken clouds,38.5167,-28.7000,
36,Hirara,JP,81.34,clear sky,24.8000,125.2833,
38,Georgetown,MY,80.89,few clouds,5.4112,100.3354,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    long = row['Lng']
    params['location'] = f'{lat},{long}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print('Hotel not found.... Skipping')

Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping
Hotel not found.... Skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

clean_hotel_df = hotel_df.copy()
clean_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

clean_hotel_df.dropna(inplace=True)


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPY_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt>
<dd>{Hotel Name}</dd>
<dt><b>City</b></dt>
<dd>{City}</dd>
<dt><b>Country</b></dt>
<dd>{Country}</dd>
<dt><b>Current Weather</b></dt>
<dd>{Current Description} & {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Call the figure to plot the data.
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))